In [230]:
import pandas as pd
import numpy as np

In [374]:
data = pd.read_csv("rome_tension.csv")
data = data[['rome5','BE_id','T']]
data = data.pivot(index='BE_id',columns='rome5',values='T')

In [382]:
run rome.py

In [127]:
def count_by(df,var):
    nb = df.groupby([var],group_keys=False).size()
    nb = pd.DataFrame({var:nb.index,'nb_'+var:nb.values})
    nb = pd.merge(df,nb,on=var,how='left')
    return nb
    
def mode_of_by(df,var1,var2):
    modes = tree.groupby([var2]).apply(lambda x: x[var1].mode()[0])
    modes = pd.DataFrame({var2:modes.index,'mode_'+var2:modes.values})
    modes = pd.merge(df,modes,on=var2,how='left')
    return modes

In [455]:
tree = pd.read_csv("fap_pcs_rome.csv")
tree.columns = 'fap','fap_lab','pcs','pcs_lab','rome','qual','rome_lab'
tree = tree.filter(items=['fap','fap_lab','rome','rome_lab'])
tree['fap']= tree['fap'].fillna(method='ffill')
tree['fap_lab']= tree['fap_lab'].fillna(method='ffill')
tree = tree.dropna(subset=['rome'],axis=0)
tree = count_by(tree,'rome')

In [447]:
multiple = tree.query('nb_rome > 1').rome.tolist()
tree['best_faps'] = tree['fap']
tree['method'] = ''
tree['n_branch'] = ''
tree['n_common'] = ''
tree['corr'] = np.nan
tree['criterium'] = ''
for rome in multiple:
    faps = tree.loc[tree.rome == rome,'fap'].tolist()
    n_branch, n_common = [0], [0]
    fap_branch, fap_corr, fap_common = '', '', ''
    tie_branch, tie_common = False, False
    sig = 0
    for fap in faps: 
        group = tree.loc[tree.fap == fap,'rome'].tolist()
        if len(group) > 1: group.remove(rome)
        corr = np.mean([data[rome].corr(data[x]) for x in group])
        branch = sum([rome[0] == x[0] for x in group])
        common = sum([x in graph[rome] for x in group])
        if corr > sig:
            sig = corr 
            fap_corr = fap
        if branch == n_branch: tie_branch = True
        if common == n_common: tie_common = True
        if branch > n_branch[-1]:
            n_branch.append(branch) 
            fap_branch = fap
            if n_branch[-2] == branch: tie = True
            else: tie_branch = False
        if common > n_common[-1]:
            n_common.append(common) 
            fap_common = fap
            if n_common[-2] == common: tie = True
            else: tie_common = False
    if sig > 0:
        best_fap = fap_corr
        method = 'corr'
        t = sig
    else:
        if n_common[-1] > 0 and tie_common == False: 
            method = 'connex'
            best_fap = fap_common
            t = n_common[-1]
        else:
            if n_branch[-1] > 0 and tie_branch == False:
                method = 'branch'
                best_fap = fap_branch
                t = n_branch[-1]
            else: 
                method = 'failed'
                best_fap = ''
                t = ''
    tree.loc[tree.rome == rome,'best_faps'] = best_fap
    tree.loc[tree.rome == rome,'method'] = method
    tree.loc[tree.rome == rome,'n_common'] = n_common[-1]
    tree.loc[tree.rome == rome,'n_branch'] = n_branch[-1]
    tree.loc[tree.rome == rome,'corr'] = sig
    tree.loc[tree.rome == rome,'criterium'] = t


In [ ]:
tree.loc[tree.method == 'failed'].sort_values(by='rome')

In [448]:
tree.loc[tree.rome == 'B1806','best_faps'] = 'F3Z40'
tree.loc[tree.rome == 'F1706','best_faps'] = 'E0Z22'

In [ ]:
tree = tree.loc[tree.fap == tree.best_faps].reset_index(drop=True)
tree = tree.filter(items=['fap','fap_lab','rome','rome_lab','method','criterium'])

In [453]:
tree.to_csv("fap_rome_correlation.csv",header=True,index=False)

,fap,fap_lab,rome,rome_lab,method,criterium
0,A0Z40,Agriculteurs salariés,A1416,"Polyculture, élevage",corr,1
1,A0Z41,Éleveurs salariés,A1403,Aide d'élevage agricole et aquacole,,
2,A0Z41,Éleveurs salariés,A1407,Élevage bovin ou équin,,
3,A0Z41,Éleveurs salariés,A1408,Élevage d'animaux sauvages ou de compagnie,,
4,A0Z41,Éleveurs salariés,A1409,Élevage de lapins et volailles,,
5,A0Z41,Éleveurs salariés,A1410,Élevage ovin ou caprin,,
6,A0Z41,Éleveurs salariés,A1411,Élevage porcin,,
7,A0Z42,"Bûcherons, sylviculteurs salariés et agents fo...",A1201,Bûcheronnage et élagage,,
8,A0Z42,"Bûcherons, sylviculteurs salariés et agents fo...",A1205,Sylviculture,,
9,A0Z43,Conducteurs d'engins agricoles ou forestiers*,A1101,Conduite d'engins d'exploitation agricole et f...,,
